## Cross-encoder re-ranking

In [1]:
from helper_utils import load_chroma, word_wrap, project_embeddings
from chromadb.utils.embedding_functions import SentenceTransformerEmbeddingFunction
import numpy as np

In [2]:
embedding_function = SentenceTransformerEmbeddingFunction()

chroma_collection = load_chroma(filename='microsoft_annual_report_2022.pdf', collection_name='microsoft_annual_report_2022', embedding_function=embedding_function)
chroma_collection.count()

C:\Users\deepa\anaconda3\envs\jkutty\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


349

# Re-ranking the long tail

In [3]:
query = "What has been the investment in research and development?"
results = chroma_collection.query(query_texts=query, n_results=10, include=['documents', 'embeddings'])

retrieved_documents = results['documents'][0]

for document in results['documents'][0]:
    print(word_wrap(document))
    print('')

• operating expenses increased $ 1. 5 billion or 14 % driven by
investments in gaming, search and news advertising, and windows
marketing. operating expenses research and development ( in millions,
except percentages ) 2022 2021 percentage change research and
development $ 24, 512 $ 20, 716 18 % as a percent of revenue 12 % 12 %
0ppt research and development expenses include payroll, employee
benefits, stock - based compensation expense, and other headcount -
related expenses associated with product development. research and
development expenses also include third - party development and
programming costs, localization costs incurred to translate software
for international markets, and the amortization of purchased software
code and services content. research and development expenses increased
$ 3. 8 billion or 18 % driven by investments in cloud engineering,
gaming, and linkedin. sales and marketing

competitive in local markets and enables us to continue to attract top
talent from ac

In [4]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

C:\Users\deepa\anaconda3\envs\jkutty\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\deepa\.cache\huggingface\hub\models--cross-encoder--ms-marco-MiniLM-L-6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

C:\Users\deepa\anaconda3\envs\jkutty\Lib\site-packages\torch\_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
pairs = [[query, doc] for doc in retrieved_documents]
scores = cross_encoder.predict(pairs)
print("Scores:")
for score in scores:
    print(score)

Scores:
0.9869336
2.6445777
-0.26802945
-10.731592
-7.706608
-5.6470013
-4.297036
-10.933232
-7.0384283
-7.3246937


In [6]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o+1)

New Ordering:
2
1
3
7
6
9
10
5
4
8


# Re-ranking with Query Expansion

In [7]:
original_query = "What were the most important factors that contributed to increases in revenue?"
generated_queries = [
    "What were the major drivers of revenue growth?",
    "Were there any new product launches that contributed to the increase in revenue?",
    "Did any changes in pricing or promotions impact the revenue growth?",
    "What were the key market trends that facilitated the increase in revenue?",
    "Did any acquisitions or partnerships contribute to the revenue growth?"
]

In [8]:
queries = [original_query] + generated_queries

results = chroma_collection.query(query_texts=queries, n_results=10, include=['documents', 'embeddings'])
retrieved_documents = results['documents']
print(retrieved_documents)

[['engineering, gaming, and linkedin. • sales and marketing expenses increased $ 1. 7 billion or 8 % driven by investments in commercial sales and linkedin. sales and marketing included a favorable foreign currency impact of 2 %. • general and administrative expenses increased $ 793 million or 16 % driven by investments in corporate functions. operating income increased $ 13. 5 billion or 19 % driven by growth across each of our segments. current year net income and diluted eps were positively impacted by the net tax benefit related to the transfer of intangible properties, which resulted in an increase to net income and diluted eps of $ 3. 3 billion and $ 0. 44, respectively. prior year net income and diluted eps were positively impacted by the net tax benefit related to the india supreme court decision on withholding taxes, which resulted in an increase to net income and diluted eps of $ 620 million and $ 0. 08, respectively.', 'independent recalculation of prices where appropriate. 

In [14]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in retrieved_documents:
    for document in documents:
        unique_documents.add(document)

unique_documents = list(unique_documents)
print(unique_documents)

['( in millions, except percentages ) 2022 2021 percentage change sales and marketing $ 21, 825 $ 20, 117 8 % as a percent of revenue 11 % 12 % ( 1 ) ppt sales and marketing expenses include payroll, employee benefits, stock - based compensation expense, and other headcount - related expenses associated with sales and marketing personnel, and the costs of advertising, promotions, trade shows, seminars, and other programs. sales and marketing expenses increased $ 1. 7 billion or 8 % driven by investments in commercial sales and linkedin. sales and marketing included a favorable foreign currency impact of 2 %. general and administrative ( in millions, except percentages ) 2022 2021 percentage change general and administrative $ 5, 900 $ 5, 107 16 % as a percent of revenue 3 % 3 % 0ppt', '37 general and administrative expenses include payroll, employee benefits, stock - based compensation expense, and other headcount - related expenses associated with finance, legal, facilities, certain h

In [15]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])
print(pairs)

[['What were the most important factors that contributed to increases in revenue?', '( in millions, except percentages ) 2022 2021 percentage change sales and marketing $ 21, 825 $ 20, 117 8 % as a percent of revenue 11 % 12 % ( 1 ) ppt sales and marketing expenses include payroll, employee benefits, stock - based compensation expense, and other headcount - related expenses associated with sales and marketing personnel, and the costs of advertising, promotions, trade shows, seminars, and other programs. sales and marketing expenses increased $ 1. 7 billion or 8 % driven by investments in commercial sales and linkedin. sales and marketing included a favorable foreign currency impact of 2 %. general and administrative ( in millions, except percentages ) 2022 2021 percentage change general and administrative $ 5, 900 $ 5, 107 16 % as a percent of revenue 3 % 3 % 0ppt'], ['What were the most important factors that contributed to increases in revenue?', '37 general and administrative expens

In [17]:
scores = cross_encoder.predict(pairs)
print(scores)

[ -7.754101   -8.505108   -5.141831  -10.042843   -7.4906554  -5.27475
  -4.8184853  -7.9171762 -11.079268  -10.711211   -3.7948625  -1.1369957
 -10.08394    -9.918428  -10.000137   -9.768024  -10.148885   -3.7681527
  -9.8078785  -6.9020905  -9.357722   -4.651892   -4.3417664]


In [12]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-7.754101
-8.505108
-5.141831
-10.042843
-7.4906554
-5.27475
-4.8184853
-7.9171762
-11.079268
-10.711211
-3.7948625
-1.1369957
-10.08394
-9.918428
-10.000137
-9.768024
-10.148885
-3.7681527
-9.8078785
-6.9020905
-9.357722
-4.651892
-4.3417664


In [13]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
    print(o)

New Ordering:
11
17
10
22
21
6
2
5
19
4
0
7
1
20
15
18
13
14
3
12
16
9
8
